# This file is intended for data preprocessing and filtering

We suggest you don't run this code unless you want to reproduce the whole dataset

In [15]:
import numpy as np
import pandas as pd

One-hot encoding anime sources and genres

In [16]:
data_anime = pd.read_csv('Data/Animelist.csv')

In [17]:
tv = data_anime[data_anime['type'] == 'TV']

In [18]:
genres = tv['genre'].str.get_dummies(sep=', ')
genres = genres.add_prefix('genre_')
genre_encoded = pd.concat([tv, genres], axis=1)

In [19]:
source_encoded = pd.get_dummies(genre_encoded['source'], prefix='source').astype(int)
data_filtered = pd.concat([genre_encoded, source_encoded], axis=1)

In [10]:
studio = data_filtered['studio'].str.get_dummies(sep=', ')
studio = genres.add_prefix('studio_')
data_filtered = pd.concat([data_filtered, studio], axis=1)

In [11]:
pd.set_option('display.max_columns', None)
data_filtered

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,airing,aired_string,aired,duration,rating,score,scored_by,rank,popularity,members,favorites,background,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,genre_Action,genre_Adventure,genre_Cars,genre_Comedy,genre_Dementia,genre_Demons,genre_Drama,genre_Ecchi,genre_Fantasy,genre_Game,genre_Harem,genre_Historical,genre_Horror,genre_Josei,genre_Kids,genre_Magic,genre_Martial Arts,genre_Mecha,genre_Military,genre_Music,genre_Mystery,genre_Parody,genre_Police,genre_Psychological,genre_Romance,genre_Samurai,genre_School,genre_Sci-Fi,genre_Seinen,genre_Shoujo,genre_Shoujo Ai,genre_Shounen,genre_Shounen Ai,genre_Slice of Life,genre_Space,genre_Sports,genre_Super Power,genre_Supernatural,genre_Thriller,genre_Vampire,source_4-koma manga,source_Book,source_Card game,source_Digital manga,source_Game,source_Light novel,source_Manga,source_Music,source_Novel,source_Original,source_Other,source_Picture book,source_Radio,source_Unknown,source_Visual novel,source_Web manga,studio_10Gauge,studio_33 Collective,studio_3xCube,studio_8bit,studio_A-1 Pictures,studio_A-Real,studio_A.C.G.T.,studio_ACC Production,studio_AIC,studio_AIC A.S.T.A.,studio_AIC Build,studio_AIC Classic,studio_AIC Frontier,studio_AIC Plus+,studio_AIC Spirits,studio_APPP,studio_AXsiZ,studio_Actas,studio_Ajia-Do,studio_Amber Film Works,studio_Amuse,studio_Animation Do,studio_Annapuru,studio_Anpro,studio_Arms,studio_Artland,studio_Artmic,studio_Asahi Production,studio_Ascension,studio_Ashi Production,studio_Asread,studio_Aubec,studio_Azeta Pictures,studio_Bandai Namco Pictures,studio_Barnum Studio,studio_BeSTACK,studio_Bee Media,studio_Bee Train,studio_Beijing Huihuang Animation Company,studio_Beijing Rocen Digital,studio_Bibury Animation Studios,studio_Blade,studio_Bones,studio_Bouncy,studio_Brain&#039;s Base,studio_Bridge,studio_C-Station,studio_C2C,studio_Calf Studio,studio_Chaos Project,studio_Charaction,studio_CloverWorks,studio_CoMix Wave Films,studio_Code,studio_Connect,studio_Cookie Jar Entertainment,studio_Craftar,studio_Creators Dot Com,studio_Creators in Pack,studio_Cyclone Graphics inc,studio_DAX Production,studio_DLE,studio_DMM.futureworks,studio_Daewon Media,studio_Dai Nippon Printing,studio_Daichi Doga,studio_DandeLion Animation Studio LLC,studio_Daume,studio_David Production,studio_Digital Frontier,studio_Digital Media Lab,studio_Diomedea,studio_Doga Kobo,studio_Dongwoo A&amp;E,studio_Dongyang Animation,studio_Dwango,studio_E&amp;G Films,studio_EKACHI EPILKA,studio_EMT²,studio_Echo,studio_Egg,studio_Egg Firm,studio_Eiken,studio_Emon,studio_Encourage Films,studio_Enoki Films,studio_FOREST Hunting One,studio_Fanworks,studio_Front Line,studio_Fuji TV,studio_Fukushima Gainax,studio_Future Planet,studio_G&amp;G Entertainment,studio_G.CMay Animation &amp; Film,studio_GANSIS,studio_GEMBA,studio_Gainax,studio_Gathering,studio_Genco,studio_Geno Studio,studio_Ginga Ya,studio_GoHands,studio_Gonzo,studio_Graphinica,studio_Group TAC,studio_Hal Film Maker,studio_Haoliners Animation League,studio_Heewon Entertainment,studio_Hoods Drifters Studio,studio_Hoods Entertainment,studio_Hotline,studio_Husio Studio,studio_ILCA,studio_Imagin,studio_Imagineer,studio_Ishimori Entertainment,studio_Issen,studio_J.C.Staff,studio_JCF,studio_Japan Taps,studio_Japan Vistec,studio_Jinnis Animation Studios,studio_Joker Films,studio_KOO-KI,studio_Kachidoki Studio,studio_Kamikaze Douga,studio_Kanaban Graphics,studio_Karaku,studio_Kazami Gakuen Koushiki Douga-bu,studio_Kenji Studio,studio_KeyEast,studio_Kinema Citrus,studio_Knack Productions,studio_Kokusai Eigasha,studio_Kyoto Animation,studio_Kyotoma,studio_LIDENFILMS,studio_LMD,studio_LandQ studios,studio_Lapin Track,studio_Larx Entertainment,studio_Lay-duce,studio_Lerche,studio_Lesprit,studio_Life Work,studio_Lilix,studio_M.S.C,studio_M2,studio_MAPPA,studio_MMDGP,studio_Madhouse,studio_Magic Bus,studio_Manglobe,studio

Filtering users to preserve ones with a valid age and location

In [21]:
data_filtered.to_csv('Data/animelist_filt.csv')

In [22]:
users = pd.read_csv('Data/UserList.csv')
users.dropna(subset = ['birth_date', 'location', 'gender'], inplace=True)
users['birth_date'] = pd.to_datetime(users['birth_date'],errors='coerce')
users['age'] = ((pd.to_datetime('2018-01-01') - users['birth_date'])/pd.Timedelta(days=365.25)).dropna().round()
users = users[users['age'] > 12] # While there is no minimum age, its unlikely that kids will have their own anime lists

In [23]:
cities = pd.read_csv('Data/worldcities.csv', usecols=['city','country','iso2','population'])
# In cases when multiple cities share the same name, we keep the ones with higher population
# Since the probability of being in Paris, France is more than 100 times higher than in Paris, Texas
cities.dropna()
cities = cities.sort_values(by='population', ascending=False)
cities = cities.drop_duplicates(subset=['city'], keep='first')

In [24]:
city_to_country = dict(zip(cities['city'].str.lower(), cities['country'].values.tolist()))
country_abbr_to_country = dict(zip(cities['iso2'].str.upper(), cities['country']))
def infer_country(location, city_to_country, country_abbr_to_country):
    location = location.lower().strip()
    
    # Check for country name in the location string
    for country in country_abbr_to_country.values():
        if country.lower() in location:
            return country
    
    # Check for country in iso format
    if location.upper() in country_abbr_to_country:
        return country_abbr_to_country[location.upper()]
    
     # Check for exact city match
    if location in city_to_country:
        return city_to_country[location]
    
    # If no match found
    return 'Unknown'

In [25]:
users['country'] = users['location'].apply(lambda loc: infer_country(loc, city_to_country, country_abbr_to_country))
users = users[users['country'] != 'Unknown']

In [26]:
users.to_csv('Data/userlist_filt.csv')
users

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,age,country
0,karthiga,2255153,3,49,1,0,0,55.31,Female,"Chennai, India",1990-04-29,NaN,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391.0,28.0,India
1,RedvelvetDaisuki,1897606,61,396,39,0,206,118.07,Female,Manila,1995-01-01,NaN,2012-12-13,1900-05-13 02:47:00,6.78,80.0,7094.0,23.0,Philippines
3,bskai,228342,25,414,2,5,11,167.16,Male,"Nayarit, Mexico",1990-12-14,NaN,2009-08-31,2014-05-12 16:35:00,8.27,1.0,10081.0,27.0,Mexico
5,terune_uzumaki,327311,5,5,0,0,0,15.20,Female,"Malaysia, Kuantan",1998-08-24,NaN,2010-05-10,2012-10-18 19:06:00,9.70,6.0,920.0,19.0,Malaysia
12,HimeAria,3129315,2,87,2,0,28,20.12,Female,Poland,1996-09-26,NaN,2013-09-08,1900-04-27 10:52:00,8.23,0.0,1314.0,21.0,Poland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302642,yoyoman9,6454558,2,32,6,7,2,24.42,Male,Boston,1999-06-27,NaN,2017-08-03,2017-12-17 11:27:00,7.50,0.0,1483.0,19.0,United States of America
302648,isoann,797785,14,213,11,20,136,65.22,Male,Poland:,1997-01-13,NaN,2011-10-23,1900-04-02 13:21:00,8.03,2.0,3944.0,21.0,Poland
302655,ammaretto,381078,0,1,1,0,0,0.49,Male,"Russia, Novokuznetsk",1990-08-02,NaN,2010-09-09,2015-06-03 10:59:00,8.00,0.0,27.0,27.0,Russia
302657,Scarlet95,2478991,6,103,10,8,54,47.94,Female,Belgium,1995-10-17,NaN,2013-04-24,2016-12-18 08:41:00,7.40,1.0,2906.0,22.0,Belgium


Making sure to drop the lists of filtered users
###### this code may take a few minutes to run

In [27]:
unique_users = users['username'].unique()
data_user_anime = pd.read_csv('Data/UserAnimeList.csv')
data_user_anime_filtered = data_user_anime[data_user_anime['username'].isin(unique_users)]

In [28]:
data_user_anime_filtered.to_csv('Data/userAnimeList_filt.csv')